# Compute the density of railspace patches

For each railspace patch, this notebook computes the percentage of neighboring railspace patches, i.e.:

```python
railspace_neighboring_patches / total_neighboring_patches
```

The neighboring patches are all patches in a user-defined radius (in meters).

In [ ]:
# solve issue with autocomplete
%config Completer.use_jedi = False

%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import pyproj
from scipy import spatial

## Inputs

In [ ]:
path2all_patches  = "XXX"
path2rail_patches = "XXX"
path2save_output  = "XXX"
# user-defined radius (in meters) for neighbors
distance_in_meters = 500

# --- example
# path2all_patches  = "./resources/all_patches_latlonpred.csv"
# path2rail_patches = "./results_v003/pred_0103_keep_1_250.csv"
# path2save_output  = "./df_pred_0103_rail_density.csv"

## Read all patches

We use this to later compute `total_neighboring_patches` in this equation: 

```python
railspace_neighboring_patches / total_neighboring_patches
```

In [ ]:
all_patches_pd = pd.read_csv(path2all_patches)

## KD-tree for all patches

In [ ]:
ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')

x, y, z = pyproj.transform(lla, ecef,
                           all_patches_pd["center_lon"].to_numpy(),
                           all_patches_pd["center_lat"].to_numpy(),
                           np.zeros(len(all_patches_pd["center_lat"])),
                           radians=False)

# add x, y, z to df
all_patches_pd["x"] = x
all_patches_pd["y"] = y
all_patches_pd["z"] = z

kdtree_patches = spatial.cKDTree(all_patches_pd[["x", "y", "z"]].to_numpy())

## Read railspace patches

In [ ]:
df_pred_0103 = pd.read_csv(path2rail_patches, index_col=0)

In [ ]:
df_pred_0103

## KD-tree for railspace patches

In [ ]:
ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')

x, y, z = pyproj.transform(lla, ecef,
                           df_pred_0103["center_lon"].to_numpy(),
                           df_pred_0103["center_lat"].to_numpy(),
                           np.zeros(len(df_pred_0103["center_lat"])),
                           radians=False)

# add x, y, z to df
df_pred_0103["x"] = x
df_pred_0103["y"] = y
df_pred_0103["z"] = z
kdtree_0103 = spatial.cKDTree(df_pred_0103[["x", "y", "z"]].to_numpy())

In [ ]:
kdtree_pred_0103 = spatial.cKDTree(df_pred_0103[["x", "y", "z"]].to_numpy())

## Calculate density

Now we can compute the terms in:

```python
railspace_neighboring_patches / total_neighboring_patches
```

In [ ]:
print("[INFO] rails.....")
railspace_neighboring_patches = kdtree_pred_0103.query_ball_tree(kdtree_pred_0103, distance_in_meters)
print("[INFO] patches...")
total_neighboring_patches     = kdtree_pred_0103.query_ball_tree(kdtree_patches, distance_in_meters)

In [ ]:
num_railspace_neighboring_patches   = [len(x) for x in railspace_neighboring_patches]
num_total_neighboring_patches       = [len(x) for x in total_neighboring_patches]

In [ ]:
df_pred_0103["perc_neigh_rails"]  = num_railspace_neighboring_patches  / (np.array(num_total_neighboring_patches) + 1e-32) * 100.

In [ ]:
df_pred_0103.iloc[0]

In [ ]:
df_pred_0103.to_csv(path2save_output)